In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from pandas.tseries.offsets import BDay

from sklearn import linear_model, neural_network, svm, cross_validation, naive_bayes, ensemble, tree, neighbors, decomposition, preprocessing

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML



import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Only Cont Fut Flag
only_cont_fut = False

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    if only_cont_fut == True:   
        if 'Fut' in exo_dict['name']:   
            exo_series = exo_series[exo_series.columns[0]]
            df[exo_dict['name']] = exo_series
            
    if only_cont_fut == False:
        exo_series = exo_series[exo_series.columns[0]]
        df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.head()

In [ ]:
px_ser = df.ES_BullishCollarBW.dropna()

In [ ]:
data = px_ser.ix['2015':'2015'].reset_index(drop=True).dropna()

In [ ]:
scatter(data.index, data.values)

In [ ]:
x = data.index.values.reshape(-1, 1)
y = data.values.reshape(-1, 1)

model = ensemble.BaggingRegressor()
model.fit(x, y)

#model.

In [ ]:
pd.concat([data, pd.Series(model.predict(x)[:,0])], axis=1)#.interpolate(method='barycentric').plot()

In [ ]:
model.predict(x)

In [ ]:
temp_df = pd.DataFrame()

temp_df['px_ser'] = px_ser

In [ ]:
reg_ser = pd.Series(model.predict(x), index=px_ser.ix['2015':'2015'].index, name='reg_ser')
#df['reg_ser'] = 
#reg_ser.drop(reg_ser.index[1:-1], inplace=True)

In [ ]:
temp_df = pd.concat([temp_df, reg_ser], axis=1)#.plot()

In [ ]:
temp_df.reg_ser.interpolate('barycentric', inplace=True)#.plot()

In [ ]:
temp_df.plot()